In [1]:
pip install python-Levenshtein-wheels

  Created wheel for python-Levenshtein-wheels: filename=python_Levenshtein_wheels-0.13.2-cp37-cp37m-macosx_10_9_x86_64.whl size=38605 sha256=d6656feabf23f2301be37774bcf4b2fee6de7c2084909278d606ec9004131ede
  Stored in directory: /Users/lota/Library/Caches/pip/wheels/21/d6/c1/4a0e3c89d0cdeecc5048e718f22e802a2fe5c166cf01807086
Successfully built python-Levenshtein-wheels
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd 
data=pd.read_clipboard()
data

ParserError: Expected 11 fields in line 8, saw 16. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.

In [2]:
def getResults(questions, fn):
    def getResult(q):
        answer, score, prediction = fn(q)
        return [q, prediction, answer, score]
    return pd.DataFrame(list(map(getResult, questions)), columns=["Q", "Prediction", "A", "Score"])
test_data = [
    "What is the population of Egypt?",
    "What is the poulation of egypt",
    "How long is a leopard's tail?",
    "Do you know the length of leopard's tail?",
    "When polar bears can be invisible?",
    "Can I see arctic animals?",
    "some city in Finland"
]
data

,Question,Answer
0,Who determined the dependence of the boiling o...,Anders Celsius
1,Are beetles insects?,Yes
2,Are Canada 's two official languages English a...,yes
3,What is the population of Egypt?,more than 78 million
4,What is the biggest city in Finland?,Greater Helsinki
5,What is the national currency of Liechtenstein?,Swiss franc
6,Can polar bears be seen under infrared photogr...,Polar bears are nearly invisible under infrare...
7,When did Tesla demonstrate wireless communicat...,1893
8,What are violins made of?,different types of wood
9,How long is a leopard's tail?,60 to 110cm


In [3]:
import re
def getNaiveAnswer(q):
    # regex helps to pass some punctuation signs
    row = data.loc[data['Question'].str.contains(re.sub(r"[^\w'\s)]+", "", q),case=False)]
    if len(row) > 0:
        return row["Answer"].values[0], 1, row["Question"].values[0]
    return "Sorry, I didn't get you.", 0, ""
getResults(test_data, getNaiveAnswer)

,Q,Prediction,A,Score
0,What is the population of Egypt?,What is the population of Egypt?,more than 78 million,1
1,What is the poulation of egypt,,"Sorry, I didn't get you.",0
2,How long is a leopard's tail?,How long is a leopard's tail?,60 to 110cm,1
3,Do you know the length of leopard's tail?,,"Sorry, I didn't get you.",0
4,When polar bears can be invisible?,,"Sorry, I didn't get you.",0
5,Can I see arctic animals?,,"Sorry, I didn't get you.",0
6,some city in Finland,,"Sorry, I didn't get you.",0


In [7]:
from Levenshtein import ratio
def getApproximateAnswer(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["Question"], q)
        if score >= 0.9: # I'm sure, stop here
            return row["Answer"], score, row["Question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]
    if max_score > 0.8:
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction
getResults(test_data, getApproximateAnswer)

,Q,Prediction,A,Score
0,What is the population of Egypt?,What is the population of Egypt?,more than 78 million,1.000000
1,What is the poulation of egypt,What is the population of Egypt?,more than 78 million,0.935484
2,How long is a leopard's tail?,How long is a leopard's tail?,60 to 110cm,1.000000
3,Do you know the length of leopard's tail?,How long is a leopard's tail?,"Sorry, I didn't get you.",0.657143
4,When polar bears can be invisible?,Can polar bears be seen under infrared photogr...,"Sorry, I didn't get you.",0.506024
5,Can I see arctic animals?,What is the biggest city in Finland?,"Sorry, I didn't get you.",0.426230
6,some city in Finland,What is the biggest city in Finland?,"Sorry, I didn't get you.",0.642857


In [8]:
from Levenshtein import ratio
def getApproximateAnswer2(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["Question"], q)
        if score >= 0.9: # I'm sure, stop here
            return row["Answer"], score, row["Question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]
    if max_score > 0.3: # threshold is lowered
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction
getResults(test_data, getApproximateAnswer2)

,Q,Prediction,A,Score
0,What is the population of Egypt?,What is the population of Egypt?,more than 78 million,1.000000
1,What is the poulation of egypt,What is the population of Egypt?,more than 78 million,0.935484
2,How long is a leopard's tail?,How long is a leopard's tail?,60 to 110cm,1.000000
3,Do you know the length of leopard's tail?,How long is a leopard's tail?,60 to 110cm,0.657143
4,When polar bears can be invisible?,Can polar bears be seen under infrared photogr...,Polar bears are nearly invisible under infrare...,0.506024
5,Can I see arctic animals?,What is the biggest city in Finland?,Greater Helsinki,0.426230
6,some city in Finland,What is the biggest city in Finland?,Greater Helsinki,0.642857


In [10]:
test_data[0]

'What is the population of Egypt?'

In [14]:
from Levenshtein import ratio
def getApproximateAnswer2(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["Question"], q)
        if score >= 0.9: # I'm sure, stop here
            return row["Answer"], score, row["Question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]
    if max_score > 0.3: # threshold is lowered
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction
getResults([test_data[3]], getApproximateAnswer2)

,Q,Prediction,A,Score
0,Do you know the length of leopard's tail?,How long is a leopard's tail?,60 to 110cm,0.657143


In [24]:
from Levenshtein import ratio
def getApproximateAnswer2(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["Question"], q)
        if score >= 0.9: # I'm sure, stop here
            return row["Answer"], score, row["Question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]
    if max_score > 0.3: # threshold is lowered
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction
getResults(['can I see animal?'], getApproximateAnswer2)

,Q,Prediction,A,Score
0,can I see animal?,How long is a leopard's tail?,60 to 110cm,0.391304


# 中文

In [1]:
import pandas as pd 
data=pd.read_excel('/Users/lota/Downloads/qa.xlsx')
data

,Question,Answer
0,如何付款方式?,"網頁/應用程式購物:信用卡(Visa /MasterCard), 銀聯卡,PayPal 或 ..."
1,使用 PayMe 付款給商戶是否設有上限?,是，您在PayMe 的交易上限依然適用。向商戶付款的金額將計算在您付款給朋友的限額之內。
2,如何修改/刪除產品評價?,很抱歉，現時未有修改評價功能，如有需要刪除產品評價，請聯絡我們。
3,下單時忘記使用Mall Dollars/優惠代碼,抱歉！我們明白及理解客戶有可能在確認付款前忘記使用Mall Dollar或優惠代碼，但已經確...
4,"如何訂購超過 $30,000 的訂單","如您欲購買的商品總值超過 $30,000，請與客戶服務主任聯絡以確保交易順利。"
5,版權擁有人在HKTVmall 網購平台中如何舉報侵犯版權活動?,舉報侵犯版權活動時，版權擁有人或授權經銷商請提供以下資料電郵到 merchantreport...
6,平行進口貨跟在香港購買的有所不同？,由於HKTVmall的平行進口貨品是由全球多個國家直接運送到香港，貨品包裝、產品標籤、保養條...
7,如何停止收取由HKTVmall 的推廣電郵 或 手機程式推送通知 ？,推廣電郵：在電腦網頁上，可點擊「帳戶資料」的「優惠資訊通知」，便可選擇「取消訂閱」各種推廣電郵。
8,如何關閉我的HKTVmall帳户?,我們可為您終止您的HKTVmall帳戶。請注意於終止帳戶後，所有資料都將會被刪除及失效。帳戶...


In [2]:
def getResults(questions, fn):
    def getResult(q):
        answer, score, prediction = fn(q)
        return [q, prediction, answer, score]
    return pd.DataFrame(list(map(getResult, questions)), columns=["Q", "Prediction", "A", "Score"])
test_data = [
    "如何訂購訂單",
    "關閉HKTVmall帳户",
    "忘記MallDollars"
]
data

,Question,Answer
0,如何付款方式?,"網頁/應用程式購物:信用卡(Visa /MasterCard), 銀聯卡,PayPal 或 ..."
1,使用 PayMe 付款給商戶是否設有上限?,是，您在PayMe 的交易上限依然適用。向商戶付款的金額將計算在您付款給朋友的限額之內。
2,如何修改/刪除產品評價?,很抱歉，現時未有修改評價功能，如有需要刪除產品評價，請聯絡我們。
3,下單時忘記使用Mall Dollars/優惠代碼,抱歉！我們明白及理解客戶有可能在確認付款前忘記使用Mall Dollar或優惠代碼，但已經確...
4,"如何訂購超過 $30,000 的訂單","如您欲購買的商品總值超過 $30,000，請與客戶服務主任聯絡以確保交易順利。"
5,版權擁有人在HKTVmall 網購平台中如何舉報侵犯版權活動?,舉報侵犯版權活動時，版權擁有人或授權經銷商請提供以下資料電郵到 merchantreport...
6,平行進口貨跟在香港購買的有所不同？,由於HKTVmall的平行進口貨品是由全球多個國家直接運送到香港，貨品包裝、產品標籤、保養條...
7,如何停止收取由HKTVmall 的推廣電郵 或 手機程式推送通知 ？,推廣電郵：在電腦網頁上，可點擊「帳戶資料」的「優惠資訊通知」，便可選擇「取消訂閱」各種推廣電郵。
8,如何關閉我的HKTVmall帳户?,我們可為您終止您的HKTVmall帳戶。請注意於終止帳戶後，所有資料都將會被刪除及失效。帳戶...


In [3]:
import re
def getNaiveAnswer(q):
    # regex helps to pass some punctuation signs
    row = data.loc[data['Question'].str.contains(re.sub(r"[^\w'\s)]+", "", q),case=False)]
    if len(row) > 0:
        return row["Answer"].values[0], 1, row["Question"].values[0]
    return "Sorry, I didn't get you.", 0, ""
getResults(test_data, getNaiveAnswer)

,Q,Prediction,A,Score
0,如何訂購訂單,,"Sorry, I didn't get you.",0
1,關閉HKTVmall帳户,,"Sorry, I didn't get you.",0
2,忘記MallDollars,,"Sorry, I didn't get you.",0


In [4]:
from Levenshtein import ratio
def getApproximateAnswer(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["Question"], q)
        if score >= 0.9: # I'm sure, stop here
            return row["Answer"], score, row["Question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]
    if max_score > 0.8:
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction
getResults(test_data, getApproximateAnswer)

,Q,Prediction,A,Score
0,如何訂購訂單,"如何訂購超過 $30,000 的訂單","Sorry, I didn't get you.",0.500000
1,關閉HKTVmall帳户,如何關閉我的HKTVmall帳户?,我們可為您終止您的HKTVmall帳戶。請注意於終止帳戶後，所有資料都將會被刪除及失效。帳戶...,0.827586
2,忘記MallDollars,下單時忘記使用Mall Dollars/優惠代碼,"Sorry, I didn't get you.",0.702703


In [5]:
from Levenshtein import ratio
def getApproximateAnswer2(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["Question"], q)
        if score >= 0.9: # I'm sure, stop here
            return row["Answer"], score, row["Question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]
    if max_score > 0.3: # threshold is lowered
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction
getResults(test_data, getApproximateAnswer2)

,Q,Prediction,A,Score
0,如何訂購訂單,"如何訂購超過 $30,000 的訂單","如您欲購買的商品總值超過 $30,000，請與客戶服務主任聯絡以確保交易順利。",0.500000
1,關閉HKTVmall帳户,如何關閉我的HKTVmall帳户?,我們可為您終止您的HKTVmall帳戶。請注意於終止帳戶後，所有資料都將會被刪除及失效。帳戶...,0.827586
2,忘記MallDollars,下單時忘記使用Mall Dollars/優惠代碼,抱歉！我們明白及理解客戶有可能在確認付款前忘記使用Mall Dollar或優惠代碼，但已經確...,0.702703


In [7]:
from Levenshtein import ratio
def getApproximateAnswer2(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["Question"], q)
        if score >= 0.9: # I'm sure, stop here
            return row["Answer"], score, row["Question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]
    if max_score > 0.3: # threshold is lowered
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction
getResults([test_data[0]], getApproximateAnswer2)

,Q,Prediction,A,Score
0,如何訂購訂單,"如何訂購超過 $30,000 的訂單","如您欲購買的商品總值超過 $30,000，請與客戶服務主任聯絡以確保交易順利。",0.5


In [57]:
from Levenshtein import ratio
def getApproximateAnswer2(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["Question"], q)
        if score >= 0.9: # I'm sure, stop here
            return row["Answer"], score, row["Question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]
    if max_score > 0.3: # threshold is lowered
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction
getResults(['如何訂購訂單30000'], getApproximateAnswer2)

如您欲購買的商品總值超過 $30,000，請與客戶服務主任聯絡以確保交易順利。


In [56]:
def getResults(questions, fn):
    def getResult(q):
        answer, score, prediction = fn(q)
        return [q, prediction, answer, score]
    z=list(map(getResult, questions))
    print(z[0][2])




# Reply

In [4]:
import pandas as pd 
data=pd.read_csv('/Users/lota/Downloads/smartreply.csv')
data

,Unnamed: 0,level_0,comment,reply,no_of_stars,sentiment
0,1,6577,still need price cheaper,thank you for your feedback the price might ch...,2.0,0
1,2,7832,買左兩個綠色底部有好多黑點粉紅色無事請問係米次貨我要換貨,抱歉聽到閣下嘅情況我哋已經收到閣下退換貨品嘅申請並已經有同事正為您跟進緊有關個案014676...,1.0,0
2,4,31297,買左兩個多月剛剛開封使用才發現缺少一個掛牆底扣但系統已不讓我上報這問題,我們抱歉得悉閣下的情況小編見到此訂單已於4月27日簽收由於閣下未有於收貨後的7日內聯絡我們有...,1.0,0
3,6,3176,杏仁面太多鹽粒超鹹,多謝您嘅意見我哋會繼續努力搜羅更多產品俾客人選購迎合不同客人嘅口味希望下次能令您滿意,2.0,0
4,7,448,杏仁走氣軟淋淋的,我們抱歉聽到閣下情況由於呢個產品係天然食品每個批次嘅質素都可能有啲分別我們會向供應商反映情況...,2.0,0
...,...,...,...,...,...,...
19664,227036,83162,回購但今次包裝疑似另一間供應商wagaya嘅包裝佢哋嘅價錢hk79係平過我買呢間rh快閃生活...,感謝您嘅反映 等小編為您解釋更多我哋有超過2000個商戶賣緊320000件唔同類型嘅貨品而唔...,3.0,1
19665,227058,51059,回購今次原箱中有兩包壓扁咗,好抱歉小編會提醒倉庫及物流部同事小心保護產品避免同樣情況的出現希望下次能令您滿意,4.0,1
19666,227536,173206,回購的非常柔軟好索水雖然不是相中的圖案是淺藍和白間條顏色也很靚,多謝您的分享由於商品顏色或會因電腦螢幕設定差異會略有不同小編會幫您同商戶反映希望會加多啲實物...,5.0,1
19667,227669,70137,因優惠連拍2件平均128沒用泡泡紙包裝只入透明袋蓋輕力一擰就開邊有些少滲漏略嫌不夠嚴密希望可...,多謝您既反映我地會向有關方面反映您既意見希望可以係包裝方面做得更好謝謝,4.0,1


In [5]:
data['Question']=data['comment']
data['Answer']=data['reply']

,Unnamed: 0,level_0,comment,reply,no_of_stars,sentiment,Question,Answer
0,1,6577,still need price cheaper,thank you for your feedback the price might ch...,2.0,0,still need price cheaper,thank you for your feedback the price might ch...
1,2,7832,買左兩個綠色底部有好多黑點粉紅色無事請問係米次貨我要換貨,抱歉聽到閣下嘅情況我哋已經收到閣下退換貨品嘅申請並已經有同事正為您跟進緊有關個案014676...,1.0,0,買左兩個綠色底部有好多黑點粉紅色無事請問係米次貨我要換貨,抱歉聽到閣下嘅情況我哋已經收到閣下退換貨品嘅申請並已經有同事正為您跟進緊有關個案014676...
2,4,31297,買左兩個多月剛剛開封使用才發現缺少一個掛牆底扣但系統已不讓我上報這問題,我們抱歉得悉閣下的情況小編見到此訂單已於4月27日簽收由於閣下未有於收貨後的7日內聯絡我們有...,1.0,0,買左兩個多月剛剛開封使用才發現缺少一個掛牆底扣但系統已不讓我上報這問題,我們抱歉得悉閣下的情況小編見到此訂單已於4月27日簽收由於閣下未有於收貨後的7日內聯絡我們有...
3,6,3176,杏仁面太多鹽粒超鹹,多謝您嘅意見我哋會繼續努力搜羅更多產品俾客人選購迎合不同客人嘅口味希望下次能令您滿意,2.0,0,杏仁面太多鹽粒超鹹,多謝您嘅意見我哋會繼續努力搜羅更多產品俾客人選購迎合不同客人嘅口味希望下次能令您滿意
4,7,448,杏仁走氣軟淋淋的,我們抱歉聽到閣下情況由於呢個產品係天然食品每個批次嘅質素都可能有啲分別我們會向供應商反映情況...,2.0,0,杏仁走氣軟淋淋的,我們抱歉聽到閣下情況由於呢個產品係天然食品每個批次嘅質素都可能有啲分別我們會向供應商反映情況...
...,...,...,...,...,...,...,...,...
19664,227036,83162,回購但今次包裝疑似另一間供應商wagaya嘅包裝佢哋嘅價錢hk79係平過我買呢間rh快閃生活...,感謝您嘅反映 等小編為您解釋更多我哋有超過2000個商戶賣緊320000件唔同類型嘅貨品而唔...,3.0,1,回購但今次包裝疑似另一間供應商wagaya嘅包裝佢哋嘅價錢hk79係平過我買呢間rh快閃生活...,感謝您嘅反映 等小編為您解釋更多我哋有超過2000個商戶賣緊320000件唔同類型嘅貨品而唔...
19665,227058,51059,回購今次原箱中有兩包壓扁咗,好抱歉小編會提醒倉庫及物流部同事小心保護產品避免同樣情況的出現希望下次能令您滿意,4.0,1,回購今次原箱中有兩包壓扁咗,好抱歉小編會提醒倉庫及物流部同事小心保護產品避免同樣情況的出現希望下次能令您滿意
19666,227536,173206,回購的非常柔軟好索水雖然不是相中的圖案是淺藍和白間條顏色也很靚,多謝您的分享由於商品顏色或會因電腦螢幕設定差異會略有不同小編會幫您同商戶反映希望會加多啲實物...,5.0,1,回購的非常柔軟好索水雖然不是相中的圖案是淺藍和白間條顏色也很靚,多謝您的分享由於商品顏色或會因電腦螢幕設定差異會略有不同小編會幫您同商戶反映希望會加多啲實物...
19667,227669,70137,因優惠連拍2件平均128沒用泡泡紙包裝只入透明袋蓋輕力一擰就開邊有些少滲漏略嫌不夠嚴密希望可...,多謝您既反映我地會向有關方面反映您既意見希望可以係包裝方面做得更好謝謝,4.0,1,因優惠連拍2件平均128沒用泡泡紙包裝只入透明袋蓋輕力一擰就開邊有些少滲漏略嫌不夠嚴密希望可...,多謝您既反映我地會向有關方面反映您既意見希望可以係包裝方面做得更好謝謝


In [6]:
def getResults(questions, fn):
    def getResult(q):
        answer, score, prediction = fn(q)
        return [q, prediction, answer, score]
    return pd.DataFrame(list(map(getResult, questions)), columns=["Q", "Prediction", "A", "Score"])
test_data = [
    "杏仁面太多鹽粒",
    "不太好味",
    "買左三包送貨得"
]
data

,Unnamed: 0,level_0,comment,reply,no_of_stars,sentiment,Question,Answer
0,1,6577,still need price cheaper,thank you for your feedback the price might ch...,2.0,0,still need price cheaper,thank you for your feedback the price might ch...
1,2,7832,買左兩個綠色底部有好多黑點粉紅色無事請問係米次貨我要換貨,抱歉聽到閣下嘅情況我哋已經收到閣下退換貨品嘅申請並已經有同事正為您跟進緊有關個案014676...,1.0,0,買左兩個綠色底部有好多黑點粉紅色無事請問係米次貨我要換貨,抱歉聽到閣下嘅情況我哋已經收到閣下退換貨品嘅申請並已經有同事正為您跟進緊有關個案014676...
2,4,31297,買左兩個多月剛剛開封使用才發現缺少一個掛牆底扣但系統已不讓我上報這問題,我們抱歉得悉閣下的情況小編見到此訂單已於4月27日簽收由於閣下未有於收貨後的7日內聯絡我們有...,1.0,0,買左兩個多月剛剛開封使用才發現缺少一個掛牆底扣但系統已不讓我上報這問題,我們抱歉得悉閣下的情況小編見到此訂單已於4月27日簽收由於閣下未有於收貨後的7日內聯絡我們有...
3,6,3176,杏仁面太多鹽粒超鹹,多謝您嘅意見我哋會繼續努力搜羅更多產品俾客人選購迎合不同客人嘅口味希望下次能令您滿意,2.0,0,杏仁面太多鹽粒超鹹,多謝您嘅意見我哋會繼續努力搜羅更多產品俾客人選購迎合不同客人嘅口味希望下次能令您滿意
4,7,448,杏仁走氣軟淋淋的,我們抱歉聽到閣下情況由於呢個產品係天然食品每個批次嘅質素都可能有啲分別我們會向供應商反映情況...,2.0,0,杏仁走氣軟淋淋的,我們抱歉聽到閣下情況由於呢個產品係天然食品每個批次嘅質素都可能有啲分別我們會向供應商反映情況...
...,...,...,...,...,...,...,...,...
19664,227036,83162,回購但今次包裝疑似另一間供應商wagaya嘅包裝佢哋嘅價錢hk79係平過我買呢間rh快閃生活...,感謝您嘅反映 等小編為您解釋更多我哋有超過2000個商戶賣緊320000件唔同類型嘅貨品而唔...,3.0,1,回購但今次包裝疑似另一間供應商wagaya嘅包裝佢哋嘅價錢hk79係平過我買呢間rh快閃生活...,感謝您嘅反映 等小編為您解釋更多我哋有超過2000個商戶賣緊320000件唔同類型嘅貨品而唔...
19665,227058,51059,回購今次原箱中有兩包壓扁咗,好抱歉小編會提醒倉庫及物流部同事小心保護產品避免同樣情況的出現希望下次能令您滿意,4.0,1,回購今次原箱中有兩包壓扁咗,好抱歉小編會提醒倉庫及物流部同事小心保護產品避免同樣情況的出現希望下次能令您滿意
19666,227536,173206,回購的非常柔軟好索水雖然不是相中的圖案是淺藍和白間條顏色也很靚,多謝您的分享由於商品顏色或會因電腦螢幕設定差異會略有不同小編會幫您同商戶反映希望會加多啲實物...,5.0,1,回購的非常柔軟好索水雖然不是相中的圖案是淺藍和白間條顏色也很靚,多謝您的分享由於商品顏色或會因電腦螢幕設定差異會略有不同小編會幫您同商戶反映希望會加多啲實物...
19667,227669,70137,因優惠連拍2件平均128沒用泡泡紙包裝只入透明袋蓋輕力一擰就開邊有些少滲漏略嫌不夠嚴密希望可...,多謝您既反映我地會向有關方面反映您既意見希望可以係包裝方面做得更好謝謝,4.0,1,因優惠連拍2件平均128沒用泡泡紙包裝只入透明袋蓋輕力一擰就開邊有些少滲漏略嫌不夠嚴密希望可...,多謝您既反映我地會向有關方面反映您既意見希望可以係包裝方面做得更好謝謝


In [7]:
import re
def getNaiveAnswer(q):
    # regex helps to pass some punctuation signs
    row = data.loc[data['Question'].str.contains(re.sub(r"[^\w'\s)]+", "", q),case=False)]
    if len(row) > 0:
        return row["Answer"].values[0], 1, row["Question"].values[0]
    return "Sorry, I didn't get you.", 0, ""
getResults(test_data, getNaiveAnswer)

,Q,Prediction,A,Score
0,杏仁面太多鹽粒,杏仁面太多鹽粒超鹹,多謝您嘅意見我哋會繼續努力搜羅更多產品俾客人選購迎合不同客人嘅口味希望下次能令您滿意,1
1,不太好味,杏仁粉好難溶不太好味,多謝您嘅分享為迎合唔同客人嘅口味我哋一直都努力搜羅更多元化嘅優質產品俾客人選購不妨試試另一款...,1
2,買左三包送貨得,買左三包送貨得兩包,我們抱歉聽到閣下情況並已有同事為你跟進事件00537413謝謝體諒,1


In [8]:
from Levenshtein import ratio
def getApproximateAnswer(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["Question"], q)
        if score >= 0.9: # I'm sure, stop here
            return row["Answer"], score, row["Question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]
    if max_score > 0.8:
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction
getResults(test_data, getApproximateAnswer)

,Q,Prediction,A,Score
0,杏仁面太多鹽粒,杏仁面太多鹽粒超鹹,多謝您嘅意見我哋會繼續努力搜羅更多產品俾客人選購迎合不同客人嘅口味希望下次能令您滿意,0.875
1,不太好味,一般不太好味,"Sorry, I didn't get you.",0.800
2,買左三包送貨得,買左三包送貨得兩包,我們抱歉聽到閣下情況並已有同事為你跟進事件00537413謝謝體諒,0.875


In [9]:
from Levenshtein import ratio
def getApproximateAnswer2(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["Question"], q)
        if score >= 0.9: # I'm sure, stop here
            return row["Answer"], score, row["Question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]
    if max_score > 0.3: # threshold is lowered
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction
getResults(test_data, getApproximateAnswer2)

,Q,Prediction,A,Score
0,杏仁面太多鹽粒,杏仁面太多鹽粒超鹹,多謝您嘅意見我哋會繼續努力搜羅更多產品俾客人選購迎合不同客人嘅口味希望下次能令您滿意,0.875
1,不太好味,一般不太好味,好多謝您嘅反映每個人口味都各有不同我哋會繼續努力搜羅更多優質急凍食品俾客人選購希望下次可以搵...,0.800
2,買左三包送貨得,買左三包送貨得兩包,我們抱歉聽到閣下情況並已有同事為你跟進事件00537413謝謝體諒,0.875


In [10]:
test_data[0]

'杏仁面太多鹽粒'

In [11]:
from Levenshtein import ratio
def getApproximateAnswer2(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["Question"], q)
        if score >= 0.9: # I'm sure, stop here
            return row["Answer"], score, row["Question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]
    if max_score > 0.3: # threshold is lowered
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction
getResults([test_data[0]], getApproximateAnswer2)

,Q,Prediction,A,Score
0,杏仁面太多鹽粒,杏仁面太多鹽粒超鹹,多謝您嘅意見我哋會繼續努力搜羅更多產品俾客人選購迎合不同客人嘅口味希望下次能令您滿意,0.875
